Todo:
- audio dataset automatic donload with code from kaggle
- make web scraping for text more flexible with names so it won't throw out an error whenever name doesnt match exactly
- do the eda with this rough data to see if it has everything we need for the pipline.
- create new file with buidling pipline so we won't have to rerun eda everytime.

# 1. Imports

## 1.1 Packages

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import os
import time
import requests
from urllib.parse import unquote

## 1.2 Audio Dataset From kaggle.com

In [3]:
audio_path = "data/audio/original_dataset/"
os.listdir(audio_path)

['38sws-dot38-caliber',
 'ak-12',
 'ak-47',
 'arctic-warfare-magnum',
 'beretta-92',
 'colt-1911',
 'daewoo-k2',
 'desert-eagle',
 'double-barrel-shotgun',
 'dp-27-lmg',
 'fn-p90',
 'fn-scar',
 'glock',
 'glock-18c',
 'glock-19-9mm-luger-pistol',
 'high-standard-22lr',
 'hk-g36c',
 'hk-ump45',
 'hk-usp-compact-40-sw-pistol',
 'kar98k',
 'kimber-45acp',
 'kriss-vector',
 'lorcin-380acp',
 'm16',
 'm24-sws',
 'm249-lmg',
 'm4',
 'mg42-lmg',
 'mini-14',
 'mk14-ebr',
 'mp-40-40-sw-pistol',
 'mp5-smg',
 'nagant-m1895',
 'ots-14-groza',
 'pp-19-bizon',
 'pump-action-shotgun',
 'qbu-88',
 'qbz-95',
 'remington-22lr',
 'remington-700',
 'remington-870',
 'rhino-60ds',
 'ruger-22lr',
 'ruger-357',
 'ruger-ar-556',
 'saiga-12k',
 'sig-p225',
 'sks-rifle',
 'slr-rifle',
 'steyr-aug',
 'sw-10-8-38spl-revolver',
 'sw-34-1-22lr-revolver',
 'thompson-m1928',
 'uzi-smg',
 'vector-lmg',
 'vss-vintorez',
 'winchester-m14',
 'winchester-rifle']

## 1.3 Image API From wikimedia.org

In [9]:
from image_api import ImageAPI
API_URL = "https://commons.wikimedia.org/w/api.php"
HEADERS = {"User-Agent": "Pipline/1.0 (mkakol.index@gmail.com)"}
audio_files = os.listdir(audio_path)
models = [os.path.splitext(f)[0] for f in audio_files]
df = ImageAPI(API_URL, HEADERS).fetch_images_for_models(models)

Searching for 38sws-dot38-caliber...
No image found for 38sws-dot38-caliber.
Searching for ak-12...
Downloaded: https://upload.wikimedia.org/wikipedia/commons/c/c2/5%2C45mm_AK-12_6P70_assault_rifle_at_Military-technical_forum_ARMY-2016_03_noBG.png
Searching for ak-47...


KeyboardInterrupt: 

## 1.4 Web Scraping Text From gunsfandom.com

In [6]:
from text_webscraping import get_fandom_text
url = "https://guns.fandom.com/wiki/AK-47"
paras = get_fandom_text(url)
for i, p in enumerate(paras[:10]):
    print(p)

<p class="sitenotice-wrapper__title">Gun Wiki</p>
<p><font face="Verdana">
</font></p>
<p><font face="Verdana"><b><span style="font-size:160%; color:#FF0000;">M.O.S.</span> —</b> <span style="font-size:120%; color:#000;">Please read and keep the <b><a href="/wiki/Gun_Wiki:Manual_of_Style" title="Gun Wiki:Manual of Style">Manual of Style</a></b> in mind while editing.</span>
</font></p>
<p><font face="Verdana"><b><span style="font-size:120%; color:#FF0000;">Also, remember that there is to be no copying and pasting from other sources, at any time - your edits must be in your OWN words!</span></b>
</font></p>
<p>READ MORE</p>
<p>
<aside class="portable-infobox pi-background pi-border-color pi-theme-wikia pi-layout-default" role="region">
<div class="pi-image-collection wds-tabber" data-source="image">
<div class="wds-tabs__wrapper">
<ul class="wds-tabs">
<li class="wds-tabs__tab wds-is-current">
<span class="wds-tabs__tab-label">
							AK-47
							
						</span>
</li>
<li class="wds-tab

In [7]:
from text_webscraping import get_fandom_text
url = "https://guns.fandom.com/wiki/colt-1911"
paras = get_fandom_text(url)
for i, p in enumerate(paras[:10]):
    print(p)

HTTPError: 404 Client Error: Not Found for url: https://guns.fandom.com/wiki/colt-1911

# 2. EDA

# 3. Text

# 6. Audio

# 5. Image

# 6... Pipline?